## Load Data

In [2]:
import pandas as pd
import random
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

#train_x = train_df.drop(columns=['PRODUCT_ID', 'Y_Class', 'TIMESTAMP'])
train_x = train_df.drop(columns=['PRODUCT_ID', 'Y_Quality', 'TIMESTAMP', 'Y_Class'])
train_y = train_df['Y_Class']

test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

## Preprocessing

In [3]:
from sklearn.preprocessing import LabelEncoder

qual_col = ['LINE','PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])

    for label in np.unique(test_x[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    
    test_x[i] = le.transform(test_x[i])

print('done')

done


In [4]:
from sklearn.model_selection import train_test_split

x_t, x_v, y_t, y_v = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

## Model

In [17]:
from xgboost import XGBClassifier # 회귀트리

xgb = XGBClassifier()
xgb.fit(x_t, y_t)
pred = xgb.predict(x_v)

from sklearn.metrics import accuracy_score

accuracy_score(pred, y_v)

[16:48:44] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_4f_b8pp6bp/croot/xgboost-split_1675119661934/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.7166666666666667

In [6]:
import sklearn.metrics
from sklearn.model_selection import train_test_split
import optuna
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

n_trials = 1000

def objective_xgbc(trial):

    xgb_max_depth = trial.suggest_int("max_depth", 2, 10, log=True)
    xgb_n_estimators = trial.suggest_int("n_estimators", 1, 50, log=True)
    xgb_learning_rate = trial.suggest_float("learning_rate", low=0.001, high=0.1, step=0.01)

    classifier_obj = XGBClassifier(max_depth=xgb_max_depth,
                                n_estimators=xgb_n_estimators,
                                learning_rate = xgb_learning_rate,
                                objective='multi:softmax')
    
    bst = classifier_obj.fit(x_t, y_t)
    preds = bst.predict(x_v)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(y_v, pred_labels)
    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgbc, n_trials=n_trials, timeout=600)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2023-02-14 17:37:18,583] A new study created in memory with name: no-name-c37f09fb-7b4d-457e-afa5-2da20f2bfd28


[17:37:18] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_4f_b8pp6bp/croot/xgboost-split_1675119661934/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2023-02-14 17:37:19,080] Trial 0 finished with value: 0.725 and parameters: {'rf_max_depth': 2, 'n_estimators': 38, 'learning_rate': 0.051000000000000004}. Best is trial 0 with value: 0.725.


[17:37:19] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_4f_b8pp6bp/croot/xgboost-split_1675119661934/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2023-02-14 17:37:20,118] Trial 1 finished with value: 0.7333333333333333 and parameters: {'rf_max_depth': 4, 'n_estimators': 46, 'learning_rate': 0.051000000000000004}. Best is trial 1 with value: 0.7333333333333333.


...

[17:47:14] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_4f_b8pp6bp/croot/xgboost-split_1675119661934/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2023-02-14 17:47:16,806] Trial 339 finished with value: 0.75 and parameters: {'rf_max_depth': 8, 'n_estimators': 46, 'learning_rate': 0.031}. Best is trial 23 with value: 0.7666666666666667.


[17:47:16] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_4f_b8pp6bp/croot/xgboost-split_1675119661934/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2023-02-14 17:47:18,900] Trial 340 finished with value: 0.7583333333333333 and parameters: {'rf_max_depth': 8, 'n_estimators': 42, 'learning_rate': 0.041}. Best is trial 23 with value: 0.7666666666666667.


Number of finished trials:  341
Best trial:
  Value: 0.7666666666666667
  Params: 
    rf_max_depth: 9
    n_estimators: 32
    learning_rate: 0.051000000000000004


In [18]:
from xgboost import XGBClassifier # 회귀트리

xgb = XGBClassifier()
xgb.fit(train_x, train_y)
pred = xgb.predict(test_x)


submit_csv = pd.read_csv('./sample_submission.csv')
submit_csv['Y_Class'] = pred
submit_csv.to_csv('XGBoost_notnull_submission.csv', index=False)

[16:49:48] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_4f_b8pp6bp/croot/xgboost-split_1675119661934/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
